In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import r2_score, classification_report, confusion_matrix, accuracy_score
from tqdm import tqdm
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/twitter-sentiment-analysis/__results__.html
/kaggle/input/twitter-sentiment-analysis/encoder.pkl
/kaggle/input/twitter-sentiment-analysis/model.h5
/kaggle/input/twitter-sentiment-analysis/__output__.json
/kaggle/input/twitter-sentiment-analysis/model.w2v
/kaggle/input/twitter-sentiment-analysis/tokenizer.pkl
/kaggle/input/twitter-sentiment-analysis/custom.css
/kaggle/input/twitter-sentiment-analysis/__results___files/__results___14_1.png
/kaggle/input/twitter-sentiment-analysis/__results___files/__results___59_0.png
/kaggle/input/twitter-sentiment-analysis/__results___files/__results___49_1.png
/kaggle/input/twitter-sentiment-analysis/__results___files/__results___49_0.png
/kaggle/input/sentiment140/training.1600000.processed.noemoticon.csv


In [2]:
cols_name = ["target", "id", "date", "flag", "user", "text"]
df = pd.read_csv(r"/kaggle/input/sentiment140/training.1600000.processed.noemoticon.csv", names = cols_name, encoding='ISO-8859-1') 
df

,target,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
...,...,...,...,...,...,...
1599995,4,2193601966,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,AmandaMarie1028,Just woke up. Having no school is the best fee...
1599996,4,2193601969,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,TheWDBoards,TheWDB.com - Very cool to hear old Walt interv...
1599997,4,2193601991,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,bpbabe,Are you ready for your MoJo Makeover? Ask me f...
1599998,4,2193602064,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,tinydiamondz,Happy 38th Birthday to my boo of alll time!!! ...


In [3]:
df.isnull().sum()

target    0
id        0
date      0
flag      0
user      0
text      0
dtype: int64

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1600000 entries, 0 to 1599999
Data columns (total 6 columns):
 #   Column  Non-Null Count    Dtype 
---  ------  --------------    ----- 
 0   target  1600000 non-null  int64 
 1   id      1600000 non-null  int64 
 2   date    1600000 non-null  object
 3   flag    1600000 non-null  object
 4   user    1600000 non-null  object
 5   text    1600000 non-null  object
dtypes: int64(2), object(4)
memory usage: 73.2+ MB


In [5]:
df.describe()

,target,id
count,1.600000e+06,1.600000e+06
mean,2.000000e+00,1.998818e+09
std,2.000001e+00,1.935761e+08
min,0.000000e+00,1.467810e+09
25%,0.000000e+00,1.956916e+09
50%,2.000000e+00,2.002102e+09
75%,4.000000e+00,2.177059e+09
max,4.000000e+00,2.329206e+09


1. 

In [6]:
df["target"].value_counts()

target
0    800000
4    800000
Name: count, dtype: int64

In [7]:
df.replace({"target" : {4:1}}, inplace = True)

In [8]:
df


,target,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
...,...,...,...,...,...,...
1599995,1,2193601966,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,AmandaMarie1028,Just woke up. Having no school is the best fee...
1599996,1,2193601969,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,TheWDBoards,TheWDB.com - Very cool to hear old Walt interv...
1599997,1,2193601991,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,bpbabe,Are you ready for your MoJo Makeover? Ask me f...
1599998,1,2193602064,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,tinydiamondz,Happy 38th Birthday to my boo of alll time!!! ...


In [9]:
port_stem = PorterStemmer()

In [10]:
def stemming(content):
    stemmed_content = re.sub("[^a-zA-Z]", "", content)
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    stemmed_content = [port_stem.stem(words) for words in stemmed_content if not words in stopwords.words("english")]
    stemmed_content = ' '.join(stemmed_content)
    
    return stemmed_content

In [11]:
df["stemmed_content"] = tqdm(df["text"].apply(stemming))

100%|██████████| 1600000/1600000 [00:00<00:00, 1881621.12it/s]


In [12]:
df["stemmed_content"]

0          switchfoothttptwitpiccomyzlawwwthatsabummeryou...
1          isupsetthathecantupdatehisfacebookbytextingita...
2          kenichanidivedmanytimesfortheballmanagedtosave...
3                       mywholebodyfeelsitchyandlikeitsonfir
4          nationwideclassnoitsnotbehavingatallimmadwhyam...
                                 ...                        
1599995           justwokeuphavingnoschoolisthebestfeelingev
1599996    thewdbcomverycooltohearoldwaltinterviewshttpbl...
1599997         areyoureadyforyourmojomakeoveraskmefordetail
1599998     happythbirthdaytomybooofallltimetupacamarushakur
1599999    happycharitytuesdaythenspccsparkscharityspeaki...
Name: stemmed_content, Length: 1600000, dtype: object

In [13]:
new_df = df
twitter_data = new_df[["target", "stemmed_content"]]
twitter_data

,target,stemmed_content
0,0,switchfoothttptwitpiccomyzlawwwthatsabummeryou...
1,0,isupsetthathecantupdatehisfacebookbytextingita...
2,0,kenichanidivedmanytimesfortheballmanagedtosave...
3,0,mywholebodyfeelsitchyandlikeitsonfir
4,0,nationwideclassnoitsnotbehavingatallimmadwhyam...
...,...,...
1599995,1,justwokeuphavingnoschoolisthebestfeelingev
1599996,1,thewdbcomverycooltohearoldwaltinterviewshttpbl...
1599997,1,areyoureadyforyourmojomakeoveraskmefordetail
1599998,1,happythbirthdaytomybooofallltimetupacamarushakur


In [14]:
X = twitter_data["stemmed_content"].values
Y = twitter_data["target"].values

In [15]:
print(X, Y)


['switchfoothttptwitpiccomyzlawwwthatsabummeryoushouldagotdavidcarrofthirddaytodoitd'
 'isupsetthathecantupdatehisfacebookbytextingitandmightcryasaresultschooltodayalsoblah'
 'kenichanidivedmanytimesfortheballmanagedtosavetherestgooutofbound' ...
 'areyoureadyforyourmojomakeoveraskmefordetail'
 'happythbirthdaytomybooofallltimetupacamarushakur'
 'happycharitytuesdaythenspccsparkscharityspeakinguphh'] [0 0 0 ... 1 1 1]


In [16]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, stratify = Y, random_state= 2)

In [17]:
print(X_train.shape, X_test.shape)

(1280000,) (320000,)


In [18]:
print(Y_train.shape, Y_test.shape)

(1280000,) (320000,)


In [19]:
vectorizer = TfidfVectorizer()

In [20]:
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

In [21]:
print(X_train)

  (0, 4269)	1.0
  (1, 370478)	1.0
  (2, 259878)	1.0
  (3, 544662)	1.0
  (4, 1125692)	1.0
  (5, 183084)	1.0
  (6, 655943)	1.0
  (7, 427969)	1.0
  (8, 372293)	1.0
  (9, 414371)	1.0
  (10, 453206)	1.0
  (11, 946311)	1.0
  (12, 1165280)	1.0
  (13, 334893)	1.0
  (14, 1029544)	1.0
  (15, 1251389)	1.0
  (16, 26694)	1.0
  (17, 620642)	1.0
  (18, 371055)	1.0
  (19, 166384)	1.0
  (20, 939661)	1.0
  (21, 392755)	1.0
  (22, 288391)	1.0
  (23, 202689)	1.0
  (24, 1253472)	1.0
  :	:
  (1279975, 97244)	1.0
  (1279976, 1110776)	1.0
  (1279977, 938969)	1.0
  (1279978, 783480)	1.0
  (1279979, 800934)	1.0
  (1279980, 865418)	1.0
  (1279981, 587370)	1.0
  (1279982, 952975)	1.0
  (1279983, 412833)	1.0
  (1279984, 81684)	1.0
  (1279985, 59066)	1.0
  (1279986, 1015971)	1.0
  (1279987, 777243)	1.0
  (1279988, 1218709)	1.0
  (1279989, 450972)	1.0
  (1279990, 465151)	1.0
  (1279991, 249222)	1.0
  (1279992, 1169847)	1.0
  (1279993, 1170137)	1.0
  (1279994, 75555)	1.0
  (1279995, 1106989)	1.0
  (1279996, 561111)	1

In [22]:
print(X_test)

  (22, 137709)	1.0
  (36, 790487)	1.0
  (55, 1232068)	1.0
  (104, 806213)	1.0
  (316, 675309)	1.0
  (317, 183110)	1.0
  (326, 530902)	1.0
  (343, 483231)	1.0
  (356, 1004786)	1.0
  (405, 294045)	1.0
  (411, 754846)	1.0
  (412, 671785)	1.0
  (503, 922733)	1.0
  (575, 484365)	1.0
  (576, 68315)	1.0
  (585, 553737)	1.0
  (597, 437993)	1.0
  (704, 437477)	1.0
  (730, 522522)	1.0
  (735, 1190472)	1.0
  (763, 201539)	1.0
  (851, 230654)	1.0
  (887, 945330)	1.0
  (905, 1080766)	1.0
  (965, 64114)	1.0
  :	:
  (319388, 469685)	1.0
  (319391, 363430)	1.0
  (319401, 69390)	1.0
  (319403, 1010168)	1.0
  (319419, 435823)	1.0
  (319442, 1154002)	1.0
  (319462, 331998)	1.0
  (319474, 363236)	1.0
  (319550, 1020181)	1.0
  (319573, 1087679)	1.0
  (319595, 377142)	1.0
  (319630, 416243)	1.0
  (319632, 324586)	1.0
  (319640, 333810)	1.0
  (319649, 564791)	1.0
  (319673, 473916)	1.0
  (319679, 363450)	1.0
  (319687, 874322)	1.0
  (319700, 937693)	1.0
  (319712, 230590)	1.0
  (319779, 553484)	1.0
  (319838

In [23]:
model = LogisticRegression(max_iter=2000)

In [24]:
model.fit(X_train, Y_train)

LogisticRegression(max_iter=2000)

In [25]:
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(Y_train, X_train_prediction)

In [26]:
print("Accuracy score on the training data: ",training_data_accuracy)

Accuracy score on the training data:  0.99812265625


In [27]:
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(Y_test, X_test_prediction)

In [28]:
print("Accuracy score on the test data: ",test_data_accuracy)

Accuracy score on the test data:  0.512653125


In [29]:
import pickle

In [30]:
filename = "trained_model.sav"
pickle.dump(model, open(filename, "wb"))

In [31]:
loaded_model = pickle.load(open('/kaggle/working/trained_model.sav', 'rb'))

In [32]:
X_new = X_test[200]
print(Y_test[200])

prediction = model.predict(X_new)
print(prediction)

if (prediction[0] == 0):
    print("Negative Tweet")
else:
    print("Positive Tweet")

1
[1]
Positive Tweet
